In [ ]:
from sepal_ui import sepalwidgets as sw

import ipyvuetify as v
from utils import messages as ms

In [ ]:
#use a class to define your input and output in order to have mutable variables
class Mspa_process_io:
    def __init__(self):
        #set up your inputs
        self.foreground_connectivity = None
        self.edge_width = None
        self.transition_core = True
        self.separate_feature = True
        self.statistics = True
        
mp_io = Mspa_process_io()

In [ ]:
#create an empty result tile that will be filled with displayable plot, map, links, text
id_ = "result_widget"
title = "Results"
mp_results = sw.Tile(id_, title, inputs=[ms.MSPA_NO_RESULTS])

In [ ]:
#create the output alert 
mp_output = sw.Alert().add_msg(ms.MSPA_MESSAGE)

#create the button that start your process
mp_btn = sw.Btn(ms.MSPA_BTN)

#create the widgets following ipyvuetify requirements 
mp_connectivity = v.Select(items=[4,8], label='Foreground connectivity',  v_model=None)
mp_edge = v.Slider(label= 'Edge width (px)', max=100, class_="mt-5", thumb_label='always', v_model=0)
mp_transition = v.Switch(label="Transition Core - Loop/Bridge", v_model=True)
mp_feature = v.Switch(label="Separate internal from external features", v_model=True)
mp_statistics = v.Switch(label="Compute statistics", v_model=True)


mp_inputs = [
    mp_connectivity, 
    mp_edge, 
    mp_transition, 
    mp_feature, 
    mp_statistics
]

#bind the widget to the inputs
mp_output.bind(mp_connectivity, mp_io, 'foreground_connectivity')
mp_output.bind(mp_edge, mp_io, 'edge_width')
mp_output.bind(mp_transition, mp_io, 'transition_core')
mp_output.bind(mp_feature, mp_io, 'separate_feature')
mp_output.bind(mp_statistics, mp_io, 'statistics')


#create a process tile
id_ = "input_widget"
title = 'Create fragmentation map'

mp_widget = sw.Tile(
    id_, 
    title, 
    btn=mp_btn, 
    inputs=mp_inputs, 
    output=mp_output
)

In [ ]:
#bind the button to the process by writing a custom function
from scripts import run
from functools import partial

def process_start(widget, event, data, output, mp_results):    
    
    #toggle the loading button
    widget.toggle_loading()

    ###################################
    ###          load inputs        ###
    ###################################

    bin_map = getattr(bi_io, 'bin_map')    
    params = [
        str(getattr(mp_io, 'foreground_connectivity')),
        str(getattr(mp_io,'edge_width')),
        str(int(getattr(mp_io, 'transition_core'))),
        str(int(getattr(mp_io, 'separate_feature'))),
        str(int(getattr(mp_io, 'statistics'))),
    ]
            
    #check inputs
    if not output.check_input(bin_map, ms.NO_MAP): return widget.toggle_loading()
    for param in params:
        if not output.check_input(param, ms.NO_INPUT): 
            return widget.toggle_loading()
    
    #launch any process you want
    #try:
    output_layout = run.mspa_analysis(
        bin_map = bin_map, 
        params  = params,
        output  = output
    )
    
    #display the image 
    mp_results.children[0].children = [mp_results.children[0].children[0]] + output_layout
    
    #except Exception as e: 
    #    output.add_live_msg(str(e), 'error')
        
    #toggle the loading button
    widget.toggle_loading()

mp_btn.on_event('click', partial(
    process_start, 
    output=mp_output,
    mp_results=mp_results
))

In [ ]:
mp_widget

In [ ]:
mp_results